In [1]:
#importing libraries
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from keras.backend import dropout
import keras
import random
from sklearn.model_selection import train_test_split

2024-03-03 19:15:14.426193: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-03 19:15:15.032538: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-03-03 19:15:15.032590: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-03-03 19:15:15.032595: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# Set a seed value
seed_value= 2

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

# 5. For layers that introduce randomness like dropout, make sure to set seed values 
# model.add(Dropout(0.25, seed=seed_value))

In [3]:
#import datasets

train = pd.read_csv("trainsplitfinal.csv")
test = pd.read_csv("testsplitfinal.csv")
val = pd.read_csv("valsplitfinal.csv")

In [4]:
#shuffle the datasets before splitting into X and Y
train = train.sample(frac=1, random_state=42)
test = test.sample(frac=1, random_state=42)
val = val.sample(frac=1, random_state=42)

In [5]:
train.groupby(["labels"]).size()

labels
L_Alanine     92
L_Arginine    99
L_Lysine      30
dtype: int64

In [6]:
test.groupby(["labels"]).size()

labels
L_Alanine     20
L_Arginine    21
L_Lysine       6
dtype: int64

In [7]:
val.groupby(["labels"]).size()

labels
L_Alanine     20
L_Arginine    22
L_Lysine       6
dtype: int64

In [8]:
#train
#one hot encode labels
df_dummies_train = pd.get_dummies(train['labels'])
df_class_neural_train = pd.concat([train,df_dummies_train],axis=1)
df_class_neural_train.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,labels,L_Alanine,L_Arginine,L_Lysine
132,0.120717,-0.276917,0.107355,0.535899,0.620908,0.753653,1.327079,1.748589,2.071298,2.264882,...,0.647858,0.509612,-0.177752,-0.449278,-0.472724,0.120717,L_Arginine,0,1,0
148,0.574981,0.416041,0.521903,0.506345,0.252327,0.665665,0.330955,0.613074,0.755007,1.051513,...,0.640606,0.826535,1.740564,1.767402,0.673315,0.574981,L_Alanine,1,0,0
93,-0.120386,0.655622,0.657779,0.649833,1.156540,0.914493,0.809047,1.059020,1.221142,1.332161,...,-0.238065,-0.019899,-0.304440,-0.106572,-0.036114,-0.120386,L_Arginine,0,1,0
180,0.346422,0.536368,0.219828,-0.323506,-0.043716,0.029544,0.013213,0.243993,0.627797,1.114473,...,1.065721,0.662993,1.033821,0.709968,0.412322,0.346422,L_Arginine,0,1,0
15,0.746837,0.447874,0.332958,-0.186665,-0.607779,0.185230,0.333004,0.220716,0.820318,0.516266,...,1.671121,1.503503,1.199286,0.498243,0.848454,0.746837,L_Arginine,0,1,0


In [9]:
#test
#one hot encode labels
df_dummies_test = pd.get_dummies(test['labels'])
df_class_neural_test = pd.concat([test,df_dummies_test],axis=1)
df_class_neural_test.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,labels,L_Alanine,L_Arginine,L_Lysine
27,1.663724,1.646142,1.450220,1.134827,0.851662,0.843144,0.744588,0.661492,0.494022,0.614293,...,1.194406,1.705976,2.039258,2.494403,1.718343,1.663724,L_Arginine,0,1,0
39,0.733524,-0.154438,-1.159142,-0.723816,-0.817864,-0.495664,0.161256,0.506738,1.028581,1.254618,...,1.427726,1.564295,0.886951,1.051125,1.049582,0.733524,L_Arginine,0,1,0
26,-0.432895,-0.006113,0.316363,0.546559,0.761573,0.743488,0.662611,0.830290,1.210904,1.576646,...,-0.265973,-0.144571,-0.238312,-0.310430,-0.329081,-0.432895,L_Arginine,0,1,0
43,0.542252,0.307520,-0.248883,-0.150706,-0.096727,0.276939,-0.322530,0.603294,1.792092,1.632584,...,0.877168,0.829054,1.744678,0.955853,0.920300,0.542252,L_Alanine,1,0,0
24,0.114051,0.031418,-0.264262,-0.886127,-0.558256,-0.778959,-0.759516,-0.228709,-0.222577,0.325565,...,0.397274,0.182710,0.028726,0.199658,0.043318,0.114051,L_Arginine,0,1,0


In [10]:
#val
#one hot encode labels
df_dummies_val = pd.get_dummies(val['labels'])
df_class_neural_val = pd.concat([val,df_dummies_val],axis=1)
df_class_neural_val.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,labels,L_Alanine,L_Arginine,L_Lysine
27,0.950241,0.224008,-0.474777,-0.599175,0.173910,0.537259,0.530933,0.857190,1.083375,0.853634,...,1.193222,1.302273,1.620642,1.267208,1.499953,0.950241,L_Alanine,1,0,0
40,1.085238,1.002057,0.826076,0.374210,0.274805,0.116663,0.491072,0.637134,0.745689,0.924217,...,0.421344,0.226861,0.854329,0.820714,0.792987,1.085238,L_Arginine,0,1,0
26,0.510472,0.142635,-0.183376,-0.341472,-0.053207,-0.542771,-0.343561,-0.120553,-0.001048,0.329413,...,1.241294,1.125350,1.040342,0.977549,1.057136,0.510472,L_Alanine,1,0,0
43,-0.620131,-0.510840,-0.125425,-0.185303,-0.214913,0.883478,0.751567,0.415057,1.741979,1.501432,...,-0.524324,-0.512558,-0.522201,-0.432157,-0.644628,-0.620131,L_Arginine,0,1,0
24,0.479989,0.150066,-0.755433,-0.249371,-0.250913,-0.360404,0.066803,0.745609,0.512684,1.036370,...,0.763469,1.246140,1.114943,0.858173,0.848272,0.479989,L_Arginine,0,1,0


In [11]:
X_train = df_class_neural_train.iloc[:,:72]
y_train = df_class_neural_train.iloc[:,73:]
X_test = df_class_neural_test.iloc[:,:72]
y_test = df_class_neural_test.iloc[:,73:]
X_val = df_class_neural_val.iloc[:,:72]
y_val = df_class_neural_val.iloc[:,73:]

In [12]:
#check shape
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(221, 72)
(221, 3)
(47, 72)
(47, 3)
(48, 72)
(48, 3)


In [13]:
#save the splits
X_train.to_csv("finalXtrain.csv",index=False)
y_train.to_csv("finalYtrain.csv",index=False)
X_test.to_csv("finalXtest.csv",index=False)
y_test.to_csv("finalYtest.csv",index=False)
X_val.to_csv("finalXval.csv",index=False)
y_val.to_csv("finalYval.csv",index=False)